In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import pymysql
from sqlalchemy import create_engine
import getpass
password = getpass.getpass()

from imblearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

········


In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

Instructions
Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.
Read the data into a Pandas dataframe.
Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.
Create a logistic regression model to predict this variable from the cleaned data.
Evaluate the results.

## Project definition

In [9]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

q = '''
SELECT f.*, c.name AS category_name, 
       q.times_rented_aug_2005, q2.times_rented_july_2005,
       q3.times_rented_june_2005, q4.times_rented_may_2005
  FROM film as f
  JOIN film_category AS fc
    ON fc.film_id = f.film_id
  JOIN category as c
    ON c.category_id = fc.category_id
  LEFT JOIN (SELECT f.film_id, COUNT(*) times_rented_aug_2005
		  FROM rental as r
		  JOIN inventory as i
			ON i.inventory_id = r.inventory_id
		  JOIN film as f
			ON f.film_id = i.film_id
		 WHERE SUBSTRING(r.rental_date, 1, 7) = "2005-08"
		 GROUP BY f.film_id) as q
	ON q.film_id = f.film_id
 LEFT JOIN (SELECT f.film_id, COUNT(*) times_rented_july_2005
		  FROM rental as r
		  JOIN inventory as i
			ON i.inventory_id = r.inventory_id
		  JOIN film as f
			ON f.film_id = i.film_id
		 WHERE SUBSTRING(r.rental_date, 1, 7) = "2005-07"
		 GROUP BY f.film_id) as q2
	ON q2.film_id = f.film_id
  LEFT JOIN (SELECT f.film_id, COUNT(*) times_rented_june_2005
		  FROM rental as r
		  JOIN inventory as i
			ON i.inventory_id = r.inventory_id
		  JOIN film as f
			ON f.film_id = i.film_id
		 WHERE SUBSTRING(r.rental_date, 1, 7) = "2005-06"
		 GROUP BY f.film_id) as q3
	ON q3.film_id = f.film_id
  LEFT JOIN (SELECT f.film_id, COUNT(*) times_rented_may_2005
		  FROM rental as r
		  JOIN inventory as i
			ON i.inventory_id = r.inventory_id
		  JOIN film as f
			ON f.film_id = i.film_id
		 WHERE SUBSTRING(r.rental_date, 1, 7) = "2005-05"
		 GROUP BY f.film_id) as q4
	ON q4.film_id = f.film_id;
'''

data = pd.read_sql_query(q, engine)

In [10]:
data

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,category_name,times_rented_aug_2005,times_rented_july_2005,times_rented_june_2005,times_rented_may_2005
0,19,AMADEUS HOLY,A Emotional Display of a Pioneer And a Technic...,2006,1,None,6,0.99,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,Action,7.0,8.0,4.0,1.0
1,21,AMERICAN CIRCUS,A Insightful Drama of a Girl And a Astronaut w...,2006,1,None,3,4.99,129,17.99,R,"Commentaries,Behind the Scenes",2006-02-15 05:03:42,Action,8.0,8.0,3.0,2.0
2,29,ANTITRUST TOMATOES,A Fateful Yarn of a Womanizer And a Feminist w...,2006,1,None,5,2.99,168,11.99,NC-17,"Trailers,Commentaries,Deleted Scenes",2006-02-15 05:03:42,Action,2.0,4.0,2.0,2.0
3,38,ARK RIDGEMONT,A Beautiful Yarn of a Pioneer And a Monkey who...,2006,1,None,6,0.99,68,25.99,NC-17,"Trailers,Commentaries,Deleted Scenes,Behind th...",2006-02-15 05:03:42,Action,NaN,NaN,NaN,NaN
4,56,BAREFOOT MANCHURIAN,A Intrepid Story of a Cat And a Student who mu...,2006,1,None,6,2.99,129,15.99,G,"Trailers,Commentaries",2006-02-15 05:03:42,Action,6.0,8.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,931,VALENTINE VANISHING,A Thrilling Display of a Husband And a Butler ...,2006,1,None,7,0.99,48,9.99,PG-13,"Trailers,Behind the Scenes",2006-02-15 05:03:42,Travel,5.0,3.0,2.0,2.0
996,977,WINDOW SIDE,A Astounding Character Study of a Womanizer An...,2006,1,None,3,2.99,85,25.99,R,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,Travel,2.0,6.0,3.0,NaN
997,981,WOLVES DESIRE,A Fast-Paced Drama of a Squirrel And a Robot w...,2006,1,None,7,0.99,55,13.99,NC-17,Behind the Scenes,2006-02-15 05:03:42,Travel,8.0,9.0,4.0,NaN
998,988,WORKER TARZAN,A Action-Packed Yarn of a Secret Agent And a T...,2006,1,None,7,2.99,139,26.99,R,"Trailers,Commentaries,Behind the Scenes",2006-02-15 05:03:42,Travel,7.0,6.0,2.0,NaN
